# Main Dataset clean

**Abstract:**
One-sentence description

**Description:**
In the following cell, I...


In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
from matplotlib import pyplot as plt

import sys
import os

sys.path.append("..")
%matplotlib inline
%config InlineBackend.figure_format = 'retina'
# plt.style.use('ggplot')
plt.rcParams["axes.facecolor"] = "white"

In [ ]:
from config import ROOT, SCHEME_87, PROVINCES_CHN2ENG

## Title

**Abstract:**
One-sentence description

**Description:**
In the following cell, I...


In [ ]:
# Introduction of the original datasets.
readme = pd.read_excel(
    os.path.join(ROOT, "data/source/Zhou et al_2020_PNAS_dataset.xlsx"),
    sheet_name="Readme",
)

# driver factors to water use changes
driver_data = pd.read_excel(
    os.path.join(ROOT, "data/source/Zhou et al_2020_PNAS_dataset.xlsx"), sheet_name="D3"
)

# 展示上述数据的数据介绍
readme_drivers = readme.loc[39:47, ["Variables", "Units", "Definition"]]
readme_drivers

**Output digestion:**

---
**Analysis:**
可以看到，用水驱动主要有以下几个变量：
- Year，年份，这些数据的年份都是从1975开始，到2013年
- PIRR，潜在的灌溉用水需求，根据文章里说，是由全球水文模型跑出来的，因此有不小的不确定性。
- AIRR，可供灌溉的淡水量，也是全球水文模型跑出来的
- WCI，灌溉装备了节水面积的灌区占灌区总面积的比例，比如滴灌、管灌、渠道硬化
- Ratio of industrial water recycling，工业用水是可以循环使用的，这是再利用的比例
- Ratio of industrial water evaporated，这才算是工业真正的耗水，也就是工业用水中的多少是被消耗掉的

对于该数据，有以下几个基本认识：

- 最小的灌溉需求都有315mm， 但灌溉可供水量是有差异的，因此有的地方可能水赤字，这个赤字有可能从地下水补给，对分析带来较大的影响。这些模型跑出来的数据对定量分析帮助有限。
- WUI直接由总用水量计算出，因此根本不能使用。
- 可以使用农业和工业的节水设施变量参与预测：节水灌溉比例、工业再利用率、工业蒸发率（耗水）

## Water Uses Values

**Abstract:**
One-sentence description

**Description:**
In the following cell, I...


In [ ]:
# water use values data and some economic factors
values_data = pd.read_excel(
    os.path.join(ROOT, "data/source/Zhou et al_2020_PNAS_dataset.xlsx"), sheet_name="D1"
)
values_data = values_data.dropna(axis=1, how="all")

# 清洗数据的头部，让两行头部变一行
change_name_dic = {}
last_item = "None"
for col in values_data:
    second_row = values_data.loc[0, col]
    if "Unnamed" in col:
        change_name_dic[col] = last_item + ": " + second_row
    else:
        if type(second_row) is str:
            change_name_dic[col] = col + ": " + second_row
        last_item = col

values_data.rename(change_name_dic, axis=1, inplace=True)
values_data = values_data.drop(0)

# 重命名表头，取消两边的空格
values_data.rename({col: col.strip() for col in values_data}, axis=1, inplace=True)

# 更改正确的数据类型
values_data = values_data.astype(float, errors="ignore")
values_data["Year"] = values_data["Year"].astype(int)
values_data.iloc[:, 2:] = values_data.iloc[:, 2:].astype(float)

# 整合省份数据
perfectures = pd.read_csv(os.path.join(ROOT, "data/source/perfectures.csv")).drop(
    ["FID", "Shaoefile_"], axis=1
)
values_data = pd.merge(
    left=values_data,
    right=perfectures,
    left_on="City_ID",
    right_on="Perfecture",
    right_index=False,
)
# values_data = values_data.replace("Neimeng", "Inner Mongolia")

values_data.head()
values_data.to_csv(os.path.join(ROOT, "data/processed/values_data.csv"))

**Output digestion:**

---
**Analysis:**
这个数据级我们相对比较熟悉了，主要包括了：
- 农业灌溉面积（包括总的，和各种作物的面积）以及每个种植作物的单位面积耗水量（通过作物耗水量计算得出）
- 多种工业的总产值（GVA），已经是去通货膨胀之后的数据，可以直接用，还有每个产业的单位产出用水量，也是通过水量计算得出的
- 城市居民用水、城市人口
- 农村居民用水、农村人口
- 农村牲畜
- 城市服务业产出

## Merge source datasets

**Abstract:**
One-sentence description

**Description:**
In the following cell, I...


In [ ]:
# 整合成 Panel 数据
agg_data = (
    values_data.groupby(["Province_n", "Year"])
    .sum()
    .reset_index()
    .rename({"Province_n": "Province"}, axis=1)
)
merged_data = pd.merge(
    how="left", left=agg_data, right=driver_data, on=["Province", "Year"]
)

for col in merged_data:
    if "WU" in col:
        merged_data.drop(col, axis=1, inplace=True)
        print("Column {} dropped.".format(col))
        # 与WUI有关的列其实都不能用来预测

merged_data.head()

jinji = (
    merged_data[merged_data["Province"].isin(["Hebei", "Tianjin"])]
    .groupby("Year")
    .sum()
    .reset_index()
)
jinji.loc[:, "Province"] = "Jinji"
jinji.shape

# 为每个单位制作一个独特的、数字的标签
province_index = {}
i = 1
for x in merged_data["Province"]:
    if x not in province_index:
        province_index[x] = i
        i += 1

province_index["Jinji"] = 32  # 天津河北作为第32个地理单元
index_province = {
    province_index[province]: province for province in province_index.keys()
}


merged_data = merged_data[~merged_data["Province"].isin(["Hebei", "Tianjin"])].append(
    jinji
)
merged_data["Index"] = merged_data["Province"].map(province_index)
merged_data = merged_data.reset_index(drop=True)


# 本研究只使用1975--2013年的数据，其它的不 care
mask = merged_data["Year"].isin(np.arange(1975, 2013))
merged_data = merged_data[mask]

print("\nShape after drop:")
merged_data.shape

**Output digestion:**

---
**Analysis:**
1. 
2. 
3. 

## Export Data

**Abstract:**
One-sentence description

**Description:**
In the following cell, I...


In [ ]:
# existed dataset.
pd.read_csv(os.path.join(ROOT, "data/processed/merged_data.csv")).shape

In [ ]:
# 导出预处理之后的数据，用以在其它工作中分析
merged_data.to_csv(os.path.join(ROOT, "data/processed/merged_data.csv"))
merged_data.shape

**Output digestion:**

---
**Analysis:**
1. 
2. 
3. 

## GDP

**Abstract:**
One-sentence description

**Description:**
In the following cell, I...


In [ ]:
gdp = pd.read_excel("../data/source/CRE_Gdp01.xlsx", header=1)
gdp["省份名称"] = gdp["省份名称"].str[:2].map(PROVINCES_CHN2ENG)
gdp.head()

In [ ]:
start, end = 1975, 2012
y = "地区生产总值"
ratio = True


def extract_y(y, start, end, ratio=False):
    china = gdp[gdp["省份名称"] == "China"].set_index("年度标识")
    others = gdp[gdp["省份名称"] != "China"]

    years = np.arange(start, end + 1)
    df = pd.DataFrame(index=years, columns=others["省份名称"].unique())
    for year in years:
        this_year = others[others["年度标识"] == year]
        ys = this_year.set_index("省份名称")[y]
        if ratio:
            ys = ys / china.loc[year, y]
        df.loc[year] = ys
    return df


gdp_test = extract_y(y, start, end)
gdp_test.to_csv("../data/processed/gdp_test.csv")

In [ ]:
used_cols = {
    "年度标识": "Year",
    "省份名称": "Province",
    "地区生产总值": "gdp",
    "地区生产总值－第一产业": "gdp-1",
    "地区生产总值－第二产业": "gdp-2",
    "地区生产总值－第三产业": "gdp-3",
    "人均地区生产总值": "gdp-avg",
}

used_gdp = pd.DataFrame()
for col in used_cols:
    used_gdp[used_cols[col]] = gdp[col]
used_gdp.head()

In [ ]:
merged_gdp = pd.merge(
    left=merged_data, right=used_gdp, how="left", on=["Year", "Province"]
).drop("Index", axis=1)
merged_gdp.to_csv("../data/processed/merged_data.csv")

## Natural context

In [ ]:
from config import NATURAL

os.chdir(ROOT)

NATURAL

In [ ]:
merged_data = merged_gdp.copy()

for var in NATURAL.keys():
    data = pd.read_csv(f"data/processed/{var}.csv", index_col=0)
    data.columns = data.columns.map(PROVINCES_CHN2ENG, na_action=None)

    dfs = []
    for col in data:
        if col is np.NaN:
            continue
        dfs.append(
            pd.DataFrame(
                {"Year": data[col].index, "Province": col, f"{var}": data[col].values}
            )
        )

    df = pd.concat(dfs)
    merged_data = pd.merge(
        left=merged_data, right=df, how="left", on=["Year", "Province"]
    )

merged_data.head()

In [ ]:
merged_data.to_csv("data/processed/merged_data.csv")